### start minie server:
```shell
mvn clean compile exec:java
```

### call example:
```shell
curl 'http://localhost:8080/minie/query' -X POST -d 'Obama visited the white house.' | jq .
```
                                                Post                data                   json

In [6]:
import requests

url = 'http://localhost:8080/minie/query'
data = 'The English people are an ethnic group and nation native to England, who speak the English language and share a common history and culture.'
r = requests.post(url, data=data)
print(r.json())

{'facts': [{'subject': 'English people', 'predicate': 'are', 'object': 'ethnic group'}, {'subject': 'English people', 'predicate': 'are', 'object': 'nation native to England'}, {'subject': 'ethnic group', 'predicate': 'speak', 'object': 'English language'}, {'subject': 'ethnic group', 'predicate': 'speak', 'object': 'share'}, {'subject': 'nation native to England', 'predicate': 'speak', 'object': 'English language'}, {'subject': 'nation native to England', 'predicate': 'speak', 'object': 'share'}]}


#### MinIE.Mode.SAFE
#### Change mode in file FaceResource.java line 30:
```java
MinIE minie = new MinIE(sentence, FactsResource.parser, MinIE.Mode.SAFE);
```

In [8]:
import pickle
import pandas as pd
import spacy
from tqdm import tqdm
url = 'http://localhost:8080/minie/query'
nlp = spacy.load('en_core_web_lg')
def prepare_text(text):
    doc=nlp(text)
    result=[]
    for sent in list(doc.sents):
            sent=str(sent)
            sent = sent.rstrip() #末尾空格
            r = requests.post(url, data=sent)
            for facts in r.json()['facts']:
                result.append({'subject':facts['subject'],'relation': facts['predicate'],'object':facts['object']})
    return result

def process(file_name):
    results = []
    var = pickle.load(open('./cleaned_data_split/'+file_name+'.sav', 'rb'))
    for text in tqdm(var):
        results += prepare_text(text)
    result = pd.DataFrame(results)
    result.to_csv('./result_minie/'+file_name+'r.csv',index=False)

In [9]:
process('a0') # a1 a2 a3 ...

  0%|                                                                                        | 0/16042 [00:06<?, ?it/s]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
var = pickle.load(open('./cleaned_data_split/a0.sav', 'rb'))

In [13]:
doc=nlp(var[0])
result=[]

In [20]:
sent=str(list(doc.sents)[0])

objective: this retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven mycoplasma pneumoniae infections at king abdulaziz university hospital, jeddah, saudi arabia.

In [23]:
sent = sent.rstrip()

In [25]:
r = requests.post(url, data=sent)

In [26]:
for facts in r.json()['facts']:
    result.append({'subject':facts['subject'],'relation': facts['predicate'],'object':facts['object']})

In [27]:
result

[{'subject': 'retrospective chart review',
  'relation': 'describes',
  'object': 'epidemiology of QUANT_O_1 patients with culture-proven mycoplasma pneumoniae infections at king abdulaziz university hospital'},
 {'subject': 'retrospective chart review',
  'relation': 'describes',
  'object': 'clinical features'},
 {'subject': 'retrospective chart review',
  'relation': 'describes',
  'object': 'epidemiology of QUANT_O_1 patients with culture-proven mycoplasma pneumoniae infections at jeddah'},
 {'subject': 'king abdulaziz university hospital',
  'relation': 'is',
  'object': 'saudi arabia'},
 {'subject': 'jeddah', 'relation': 'is', 'object': 'saudi arabia'}]

In [28]:
from stanza.server import CoreNLPClient

In [30]:
result2=[]
with CoreNLPClient(be_quite=False, annotators=["openie"], memory='16G', endpoint='http://localhost:1234', threads=12) as client:
    ann = client.annotate(sent,output_format='json')
    for sentence in ann['sentences']:
        for triple in sentence['openie']:
            if triple:
                result2.append({'subject':triple['subject'],'relation': triple["relation"],'object':triple['object']})

2021-10-07 15:53:22 INFO: Writing properties to tmp file: corenlp_server-b83eb1fa97e540b1.props
2021-10-07 15:53:22 INFO: Starting server with command: java -Xmx16G -cp C:\Users\pross\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 1234 -timeout 60000 -threads 12 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-b83eb1fa97e540b1.props -annotators openie -preload -outputFormat serialized


In [31]:
result2

[{'subject': 'retrospective chart review',
  'relation': 'epidemiology of',
  'object': '40 patients with culture proven mycoplasma pneumoniae infections at king abdulaziz university hospital'},
 {'subject': 'chart review',
  'relation': 'epidemiology of',
  'object': '40 patients with mycoplasma pneumoniae infections at king university hospital'},
 {'subject': 'retrospective chart review',
  'relation': 'describes',
  'object': 'epidemiology of 40 patients with mycoplasma pneumoniae infections'},
 {'subject': 'retrospective chart review',
  'relation': 'describes',
  'object': 'epidemiology of 40 patients with culture proven mycoplasma pneumoniae infections'},
 {'subject': 'retrospective chart review',
  'relation': 'epidemiology of',
  'object': '40 patients with proven mycoplasma pneumoniae infections'},
 {'subject': 'retrospective chart review',
  'relation': 'describes',
  'object': 'epidemiology of 40 patients with mycoplasma pneumoniae infections at king university hospital'},
 